In [ ]:
archivo_grafo = "gc_1000_1"

grafoRDD = sc.textFile(archivo_grafo)
header = grafoRDD.take(1)[0]
cant_nodos = header.split(" ")[0]

aristas = grafoRDD.filter(lambda linea: linea != header)
aristas = aristas.map(lambda arista: arista.split(" "))
aristas = aristas.flatMap(lambda arista: [arista, [arista[1], arista[0]]])


vecinos = aristas.groupByKey().mapValues(list)

color = dict( vecinos.map(lambda x: (x[0], 0)).collect() )

vecinos = vecinos.collect()

for nodo_vecinos in vecinos:
    nodo = nodo_vecinos[0]
    print "nodo", nodo
    vecinos_nodo = nodo_vecinos[1]
    
    vecinosRDD = sc.parallelize(vecinos_nodo)
    color[nodo] = 1
    
    vecinos_igual_color = vecinosRDD.filter(lambda vecino: color[vecino] == color[nodo])

    while(vecinos_igual_color.isEmpty() != True):
        color[nodo] +=1
        vecinos_igual_color = vecinosRDD.filter(lambda vecino: color[vecino] == color[nodo])

cant_colores = color[max(color, key = color.get)]


In [286]:
# Creacion del archivo de salida

archivo = open(archivo_grafo + "-resultados", 'w')
archivo.write( str(cant_colores) + "\n" )
for nodo in xrange(int(cant_nodos)):
    nodo = str(nodo)
    archivo.write( str(color[nodo]) + "\n" )

archivo.close()

In [ ]:
for nodo_vecinos in vecinos:
    nodo = nodo_vecinos[0]
    vecinos_nodo = nodo_vecinos[1]
    for vecino in vecinos_nodo:
        if color[nodo]==color[vecino]:
            print "Error"

print "Ok"